In [1]:
import os
import numpy as np
import torch
from PIL import Image
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

import gc
import numpy as np
import pandas as pd
import torch
import os
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import ConcatDataset, DataLoader, Subset, Dataset
from torchvision.datasets import DatasetFolder, VisionDataset
# This is for the progress bar.
from tqdm.auto import tqdm
import random
from pathlib import Path
import math
import cv2


In [2]:
!git clone https://github.com/pytorch/vision.git
%cd vision
# !git checkout v0.8.2

!cp references/detection/utils.py ../
!cp references/detection/transforms.py ../
!cp references/detection/coco_eval.py ../
!cp references/detection/engine.py ../
!cp references/detection/coco_utils.py ../

%cd ..
!rm -r vision
!pip install pycocotools


Cloning into 'vision'...
remote: Enumerating objects: 339088, done.
remote: Counting objects: 100% (55320/55320), done.
remote: Compressing objects: 100% (1324/1324), done.
remote: Total 339088 (delta 54547), reused 54617 (delta 53965), pack-reused 283768
Receiving objects: 100% (339088/339088), 680.27 MiB | 22.80 MiB/s, done.
Resolving deltas: 100% (312398/312398), done.
/kaggle/working/vision
/kaggle/working
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pycocotools: filename=pycocotools-2.0.6-cp310-cp310-linux_x86_64.whl size=93512 sha256=104568fc429b02d8b6ba9374fa68c59de602116506bbfc185ff520c15c086d15
  Stored in directory: /root/.cache/pip/wheels/58/e6/f9/f87c8f8be098b51b616871315318329cae12cdb618f4caac93
Successfully built pycocotools


In [3]:
def get_model_instance_segmentation(num_classes):
    # load an instance segmentation model pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(weights="DEFAULT")
    # return model
    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 1024
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)
    
    return model

In [4]:
import transforms as T

def get_transform(train):
    transforms = []
    transforms.append(T.PILToTensor())
    transforms.append(T.ConvertImageDtype(torch.float))
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
        transforms.append(T.RandomZoomOut(side_range = (1,1.5)))
        transforms.append(T.RandomPhotometricDistort())
    return T.Compose(transforms)

In [5]:
def get_valid_data(labels):
    valid_data = []
    for i in range(len(labels)):
        for j in range(len(labels.iloc[i]['annotations'])):
            if labels.iloc[i]['annotations'][j]['type'] == "blood_vessel":
                valid_data.append(labels.iloc[i]['id'])
                break
    
    return valid_data

def gen_mask(annotations):
    boxes = []
    masks = []
    # area = []
    for i in range(len(annotations)):

        if annotations[i]['type'] == "blood_vessel":
            pts = np.array(annotations[i]['coordinates'])
            # print(pts[0][0])
            min_xy = np.min(pts, axis=1)[0]
            max_xy = np.max(pts, axis=1)[0]

            boxes += [np.concatenate((min_xy, max_xy), axis=0)]

            mask = np.zeros((512,512), dtype=np.uint8)
            cv2.fillPoly(mask, pts, 1)

            # area.append((max_xy[0] - min_xy[0]) * (max_xy[1] - min_xy[1]))

            masks += [mask]
        # break
    boxes = torch.as_tensor(boxes, dtype=torch.float32)
    masks = torch.as_tensor(masks, dtype=torch.uint8)
    area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])

    return boxes , masks , area

In [6]:
class KidneyDataset(Dataset):
    def __init__(self , labels , metadata , image_list , tfm):
        self.labels = labels
        self.matadata = metadata
        self.image_list = image_list
        self.tfm = tfm
        
        self.image_with_target = {}

        image_path = Path(image_list[0]).parent
        print(image_path)
        
        for idx , col in labels.iterrows():
            
            # if f"{image_path}\\{col['id']}.tif" in image_list:
            if f"{image_path}/{col['id']}.tif" in image_list:
                polygons = col['annotations']
                # print(polygons[0])
                boxes , masks , area = gen_mask(polygons)
                iscrowd = torch.zeros((len(boxes),), dtype=torch.int64)
                
                target = {}
                target["boxes"] = boxes
                target["labels"] = torch.ones((len(boxes),), dtype=torch.int64)
                target["masks"] = masks
                target["image_id"] = torch.tensor([idx])
                target["area"] = area
                target["iscrowd"] = iscrowd
                self.image_with_target[col['id']] = target

        # print()

    def __len__(self):
        return len(self.image_list)
    
    def __getitem__(self,idx):
        
        # id = self.image_list[idx].split('\\')[-1].split('.')[0]
        id = self.image_list[idx].split('/')[-1].split('.')[0]
        img = Image.open(self.image_list[idx])
        target = self.image_with_target[id]


        if self.tfm is not None:
            img, target = self.tfm(img, target)
        # print(target)
        return img , target
        

In [7]:
ROOT = Path("/kaggle/input/hubmap-hacking-the-human-vasculature")
polygon_file = "polygons.jsonl"
image_folder = "train"
metadata_file = "tile_meta.csv"
train_valid_ratio = 0.9

labels = pd.read_json(ROOT / polygon_file , lines=True)
id_list = get_valid_data(labels)
metadata = pd.read_csv(ROOT / metadata_file)
image_list = [str(i) for i  in (ROOT / image_folder).glob('*.tif') if i.stem in id_list]
train_list = image_list[:int(len(image_list)*train_valid_ratio)]
valid_list = image_list[int(len(image_list)*train_valid_ratio):]

In [74]:
import matplotlib.pyplot as plt

def draw_fig(train , valid , name):
    print(train)
    print(valid)
    plt.plot([*range(1,len(train)+1)] , train , label = "training")
    
    if valid != None:
        plt.plot([*range(1,len(valid)+1)] , valid , label = "validation")

    plt.xticks(np.arange(0, len(train)+1, 5))
    plt.legend(loc="upper left")

    plt.savefig(f'{name}.png')
    plt.show()



In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# Initialize a model, and put it on the device specified.
model = get_model_instance_segmentation(2).to(device)
batch_size = 4
num_epochs = 20
patience = 10
optimizer = torch.optim.SGD(model.parameters(), lr=0.0003, weight_decay=1e-5)
_exp_name = "mask-rcnn-model"
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer , step_size=3 , gamma=0.1)

Downloading: "https://download.pytorch.org/models/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth" to /root/.cache/torch/hub/checkpoints/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth
100%|██████████| 170M/170M [00:02<00:00, 84.8MB/s] 


In [10]:
import utils
train_set = KidneyDataset(labels , metadata , train_list , get_transform(train=True))
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True , collate_fn=utils.collate_fn)
valid_set = KidneyDataset(labels , metadata , valid_list , get_transform(train=False))
valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True , collate_fn=utils.collate_fn)
# valid_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)
# trainset[0]


/kaggle/input/hubmap-hacking-the-human-vasculature/train


/tmp/ipykernel_29/2335065411.py:32: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /usr/local/src/pytorch/torch/csrc/utils/tensor_new.cpp:245.)
  boxes = torch.as_tensor(boxes, dtype=torch.float32)


/kaggle/input/hubmap-hacking-the-human-vasculature/train


In [71]:

from engine import train_one_epoch, evaluate

train_loss = []
valid_bbox_acc = []
valid_segm_acc = []
best = 0

for epoch in range(num_epochs):
    # train for one epoch, printing every 10 iterations
    logger = train_one_epoch(model, optimizer, train_loader , device, epoch, print_freq=10)
    train_loss.append(logger.meters['loss'].global_avg)
    

    # update the learning rate
    lr_scheduler.step()
    valid_logger = evaluate(model, valid_loader, device=device)
    
    bbox = valid_logger.coco_eval['bbox'].stats.mean()
    segm = valid_logger.coco_eval['segm'].stats.mean()
    
    valid_bbox_acc.append(bbox)
    valid_segm_acc.append(segm)
    
    if segm + bbox > best:
        best = segm + bbox
        torch.save(model.state_dict(), f"{_exp_name}_best.ckpt")
        print(f"Update Best in epoch {epoch} , bbox = {bbox} , segm = {segm}")
        
draw_fig(train_loss , None , "loss")
draw_fig(valid_bbox_acc , valid_segm_acc , "acc")

Epoch: [0]  [0/5]  eta: 0:00:01  lr: 0.000000  loss: 3.2330 (3.2330)  loss_classifier: 0.3299 (0.3299)  loss_box_reg: 0.0865 (0.0865)  loss_mask: 0.9912 (0.9912)  loss_objectness: 1.7419 (1.7419)  loss_rpn_box_reg: 0.0834 (0.0834)  time: 0.2726  data: 0.0224  max mem: 3309
Epoch: [0]  [4/5]  eta: 0:00:00  lr: 0.000000  loss: 2.3476 (2.4173)  loss_classifier: 0.3351 (0.3413)  loss_box_reg: 0.1738 (0.1812)  loss_mask: 1.1108 (1.0504)  loss_objectness: 0.5550 (0.7919)  loss_rpn_box_reg: 0.0520 (0.0526)  time: 0.2713  data: 0.0203  max mem: 3309
Epoch: [0] Total time: 0:00:01 (0.2717 s / it)
creating index...
index created!
Test:  [ 0/82]  eta: 0:00:56  model_time: 0.3875 (0.3875)  evaluator_time: 0.2813 (0.2813)  time: 0.6881  data: 0.0185  max mem: 3309
Test:  [81/82]  eta: 0:00:00  model_time: 0.3919 (0.3938)  evaluator_time: 0.2892 (0.2947)  time: 0.7345  data: 0.0202  max mem: 3309
Test: Total time: 0:00:59 (0.7216 s / it)
Averaged stats: model_time: 0.3919 (0.3938)  evaluator_time: 0

NameError: name 'draw_fig' is not defined

In [13]:
# print(pred[0]['masks'].shape)
# Image.fromarray(pred[0]['masks'][0,0].mul(255).byte().cpu().numpy())